In [10]:
# Import Modules
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

#modeling tools
import statsmodels.api as sm

from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier

%matplotlib inline
plt.style.use('dark_background')

%load_ext autoreload
%autoreload 2

pd.set_option('display.min_row', 25)
pd.set_option('display.max_column', 100)
pd.set_option('display.max_colwidth', 300)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
streets = pd.read_csv('../data/Seattle_Streets.csv', index_col='OBJECTID')

In [130]:
streets.head(2)

,ARTCLASS,COMPKEY,UNITID,UNITID2,UNITIDSORT,UNITDESC,STNAME_ORD,XSTRLO,XSTRHI,ARTDESCRIPT,OWNER,STATUS,BLOCKNBR,SPEEDLIMIT,SEGDIR,ONEWAY,ONEWAYDIR,FLOW,SEGLENGTH,SURFACEWIDTH,SURFACETYPE_1,SURFACETYPE_2,INTRLO,DIRLO,INTKEYLO,INTRHI,DIRHI,NATIONHWYSYS,STREETTYPE,PVMTCONDINDX1,PVMTCONDINDX2,TRANCLASS,TRANDESCRIPT,SLOPE_PCT,PVMTCATEGORY,PARKBOULEVARD,SHAPE_Length
OBJECTID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,2.0,1006,10.0,120.0,100120.0,1ST AVE BETWEEN SENECA ST AND UNIVERSITY ST,1ST AVE,SENECA ST,UNIVERSITY ST,Minor Arterial,,INSVC,1200.0,25.0,NW,N,,,306.0,48.0,PCC,AC/PCC,1ST AVE AND SENECA ST,NW,29611.0,1ST AVE AND UNIVERSITY ST,SE,N,Downtown Neighborhood,87.0,62.0,1,PRINCIPAL TRANSIT ROUTE,4.0,ART,N,305.966050
2,2.0,1009,10.0,150.0,100150.0,1ST AVE BETWEEN PIKE ST AND PINE ST,1ST AVE,PIKE ST,PINE ST,Minor Arterial,,INSVC,1500.0,25.0,NW,N,,,426.0,104.0,AC/PCC,PCC,1ST AVE AND PIKE ST,NW,29593.0,1ST AVE AND PINE ST,SE,N,Downtown Neighborhood,57.0,58.0,1,PRINCIPAL TRANSIT ROUTE,5.0,ART,N,426.031562


In [105]:
intersections = pd.read_csv('../data/Intersections.csv')

In [131]:
intersections.head(2)

,X,Y,OBJECTID,INTR_ID,GIS_XCOORD,GIS_YCOORD,COMPKEY,COMPTYPE,UNITID,SUBAREA,UNITDESC,ARTERIALCLASSCD,SIGNAL_MAINT_DIST,SIGNAL_TYPE,SHAPE_LNG,SHAPE_LAT
0,1.270709e+06,194387.955195,1,18213,1.270709e+06,194387.95532,340313,13,78852,GRDWM,4TH AVE S AND S HENDERSON N ST,0.0,,NONE,-122.329732,47.523051
1,1.282582e+06,234414.695012,2,10302,1.282582e+06,234414.69508,157936,13,32854,E,WOODROW PL E AND E GARFIELD ST,0.0,,NONE,-122.284745,47.633387


In [16]:
collisions = pd.read_csv('../data/feateng_collisions.csv', index_col='Datetime', low_memory=False)

In [132]:
collisions.head(2)

,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,LOCATION,EXCEPTRSNCODE,EXCEPTRSNDESC,SEVERITYCODE,SEVERITYDESC,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INJURIES,SERIOUSINJURIES,FATALITIES,INCDATE,JUNCTIONTYPE,SDOT_COLCODE,SDOT_COLDESC,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR,latlon_tup,bikerack_dist,radar_dist,t_circles_dist,marked_cross_dist
Datetime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2003-10-06 00:00:00,-122.320755,47.608073,909,3663,3663,1787335,Matched,Intersection,29797.0,BROADWAY AND CHERRY ST,NaN,NaN,0,Unknown,NaN,0,0,0,0,0,0,0,2003/10/06 00:00:00+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,3279003.0,NaN,NaN,NaN,0,0,N,"(47.608073467378794, -122.32075479129101)",0.011293,1.786588,0.248892,0.004166
2004-01-01 00:00:00,-122.316863,47.615847,18457,32216,32216,1775182,Matched,Block,NaN,12TH AVE BETWEEN E PINE ST AND E OLIVE ST,NaN,NaN,1,Property Damage Only Collision,Other,5,0,0,2,0,0,0,2004/01/01 00:00:00+00,Mid-Block (but intersection related),16.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, LEFT SIDE SIDESWIPE",NaN,0,Raining,Dry,Dark - Street Lights On,NaN,4001022.0,NaN,23,From same direction - all others,0,0,N,"(47.6158466942628, -122.31686348709101)",0.008300,1.251186,0.073032,0.034031


In [41]:
collisions[collisions['SEVERITYCODE'].isin(['2b', '2'])].groupby(['SEVERITYCODE', 'SEVERITYDESC']).count()

,,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,LOCATION,EXCEPTRSNCODE,EXCEPTRSNDESC,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INJURIES,SERIOUSINJURIES,FATALITIES,INCDATE,JUNCTIONTYPE,SDOT_COLCODE,SDOT_COLDESC,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR,latlon_tup,bikerack_dist,radar_dist,t_circles_dist,marked_cross_dist
SEVERITYCODE,SEVERITYDESC,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,Injury Collision,58441,58441,58441,58441,58441,58441,58441,58250,27919,58168,27301,515,57480,58441,58441,58441,58441,58441,58441,58441,58441,58039,58441,58441,10400,57492,57435,57461,57429,4229,33507,3561,58428,57480,58441,58441,58441,58441,58441,58441,58441,58441
2b,Serious Injury Collision,3077,3077,3077,3077,3077,3077,3077,3074,1552,3069,1393,18,3037,3077,3077,3077,3077,3077,3077,3077,3077,3059,3077,3077,361,3037,3031,3036,3030,430,1869,373,3077,3037,3077,3077,3077,3077,3077,3077,3077,3077


In [42]:
collisions.groupby(['SEVERITYCODE', 'SEVERITYDESC']).count()

,,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,LOCATION,EXCEPTRSNCODE,EXCEPTRSNDESC,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INJURIES,SERIOUSINJURIES,FATALITIES,INCDATE,JUNCTIONTYPE,SDOT_COLCODE,SDOT_COLDESC,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR,latlon_tup,bikerack_dist,radar_dist,t_circles_dist,marked_cross_dist
SEVERITYCODE,SEVERITYDESC,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,Unknown,21509,21509,21509,21509,21509,21509,21509,19733,4463,19623,12735,6073,1,21509,21509,21509,21509,21509,21509,21509,21509,15939,21508,21508,0,0,1,1,1,1,10205,0,12115,1,21509,21509,21509,21509,21509,21509,21509,21509
1,Property Damage Only Collision,136964,136964,136964,136964,136964,136964,136964,135228,37381,134556,58242,5139,133330,136964,136964,136964,136964,136964,136964,136964,136964,131021,136964,136964,19410,133338,133192,133241,133110,464,81429,5847,136959,133330,136964,136964,136964,136964,136964,136964,136964,136964
2,Injury Collision,58441,58441,58441,58441,58441,58441,58441,58250,27919,58168,27301,515,57480,58441,58441,58441,58441,58441,58441,58441,58441,58039,58441,58441,10400,57492,57435,57461,57429,4229,33507,3561,58428,57480,58441,58441,58441,58441,58441,58441,58441,58441
2b,Serious Injury Collision,3077,3077,3077,3077,3077,3077,3077,3074,1552,3069,1393,18,3037,3077,3077,3077,3077,3077,3077,3077,3077,3059,3077,3077,361,3037,3031,3036,3030,430,1869,373,3077,3037,3077,3077,3077,3077,3077,3077,3077,3077
3,Fatality Collision,346,346,346,346,346,346,346,346,156,346,263,0,338,346,346,346,346,346,346,346,346,345,346,346,16,338,338,338,338,53,195,89,346,338,346,346,346,346,346,346,346,346


In [46]:
# replace 2b with 3 and 3 with 4 in order to change column into int type
collisions['SEVERITYCODE'].replace('3', '4', inplace=True)
collisions['SEVERITYCODE'].replace('2b', '3', inplace=True)

In [47]:
collisions['SEVERITYCODE'].value_counts()

1    136964
2     58441
0     21509
3      3077
4       346
Name: SEVERITYCODE, dtype: int64

In [55]:
# Severity code only contains one nan value and given that the severity desc is Unknown, the appropriate label is 0
collisions['SEVERITYCODE'].fillna(value=0, inplace=True)

In [60]:
collisions['SEVERITYCODE'] = collisions['SEVERITYCODE'].astype(int)

In [61]:
collisions[['SEVERITYCODE', 'INJURIES', 'SERIOUSINJURIES', 'FATALITIES']].corr()

,SEVERITYCODE,INJURIES,SERIOUSINJURIES,FATALITIES
SEVERITYCODE,1.000000,0.700327,0.280234,0.168116
INJURIES,0.700327,1.000000,0.279734,0.067403
SERIOUSINJURIES,0.280234,0.279734,1.000000,0.173780
FATALITIES,0.168116,0.067403,0.173780,1.000000


In [63]:
collisions[['SEVERITYCODE', 'INJURIES', 'SERIOUSINJURIES', 'FATALITIES']].head()

,SEVERITYCODE,INJURIES,SERIOUSINJURIES,FATALITIES
Datetime,,,,
2003-10-06 00:00:00,0,0,0,0
2004-01-01 00:00:00,1,0,0,0
2004-01-01 00:00:00,2,1,0,0
2004-01-01 00:00:00,2,2,0,0
2004-01-01 00:00:00,2,1,0,0


Future predictions will be given only information about the intersection, so choose columns that would have information without an accident happening